This notebook was created to test ways to vectorize some of the code

In [1]:
import os
import pandas as pd
import numpy as np

#from sgp4.functions import jday
#from sgp4.conveniences import jday_datetime
from datetime import datetime, timezone

In [4]:
df = pd.read_pickle(os.environ['my_home_path'] + '/data/space-track-gp-hist-sample/raw_compiled/train.pkl')

In [5]:
def create_index_map1(df):
    '''
    This will create a map between an input record (for X_train) and
    a label record (for y_train) that will be used by the pytorch
    dataset class to dynamically build a dataset without taking up
    more space than is necessary.
    '''
    
    # ML Structure
    # Input:
    #  - Reference TLE Data (+ EPOCH)
    #  - Target EPOCH
    # Output:
    #  - Target TLE Data
    
    def groups(lst):
        arr = lst.copy()
        np.random.shuffle(arr)
        i=1
        if len(lst)<=1:
            return
        while True:
            if i==len(lst):
                yield tuple((arr[i-1],arr[0]))
                break
            else:
                yield tuple((arr[i-1],arr[i]))
                i+=1
    
    # For each unique NORAD, find all TLE indexes and generate
    # a list of combinations
    idx_pairs = []
    for norad in df['NORAD_CAT_ID'].unique():
        norad_idxs = df[df['NORAD_CAT_ID']==norad].index.values
        if len(norad_idxs > 1):
            idx_pairs.extend(groups(norad_idxs))
    idx_pairs = np.array(idx_pairs)
    
    return idx_pairs

%time create_index_map1(df)

Wall time: 1.44 s


array([[ 14167,  51528],
       [ 51528,  97678],
       [ 97678,  30861],
       ...,
       [102771,  51188],
       [ 77572,  34812],
       [ 34812,  77572]], dtype=int64)

In [15]:
np.array

<function numpy.array>

In [18]:
from collections import defaultdict
from functools import partial

def groups(lst):
    arr = lst.copy()
    np.random.shuffle(arr)
    i=1
    if len(lst)<=1:
        return
    while True:
        if i==len(lst):
            yield tuple((arr[i-1],arr[0]))
            break
        else:
            yield tuple((arr[i-1],arr[i]))
            i+=1

norad_idx = defaultdict(partial(np.ndarray, 0))
for i,n in df.reset_index()[['index','NORAD_CAT_ID']].to_numpy():
    np.append(norad_idx[n], i)
norad_idx

defaultdict(functools.partial(<class 'numpy.ndarray'>, 0),
            {18549: array([], dtype=float64),
             18727: array([], dtype=float64),
             19027: array([], dtype=float64),
             19128: array([], dtype=float64),
             19242: array([], dtype=float64),
             19425: array([], dtype=float64),
             19640: array([], dtype=float64),
             20797: array([], dtype=float64),
             6065: array([], dtype=float64),
             21294: array([], dtype=float64),
             21344: array([], dtype=float64),
             21388: array([], dtype=float64),
             21469: array([], dtype=float64),
             333: array([], dtype=float64),
             17233: array([], dtype=float64),
             17018: array([], dtype=float64),
             20100: array([], dtype=float64),
             20359: array([], dtype=float64),
             23261: array([], dtype=float64),
             23263: array([], dtype=float64),
             2989: array